# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import seaborn as sns
import math

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
import joblib
from sklearn.preprocessing import StandardScaler

## Chargement des données

In [ ]:
# Chargement depuis parquet
df = pd.read_parquet("../data/clean/lille_2022.parquet")

# Filtrer biens avec exactement 4 pièces principales
df_4p = df[df['Nombre pieces principales'] == 4].copy()

# garder les maisons et appartements
df_4p = df_4p[df_4p['Code type local'].isin([1, 2])].copy()

## préparation des données immobilières


#### `colonnes_a_garder`:
- Liste les colonnes utiles pour l’analyse des logements (surface, nombre de pièces, valeur, type).

#### `df_logements = df_4p[colonnes_a_garder].copy()`:
- Crée un nouveau DataFrame avec uniquement ces colonnes, afin de travailler sur un sous-ensemble propre.

#### Calcul de prix_m2 pondéré:
- Le prix au m² est ici calculé en prenant en compte la surface bâtie mais aussi une part pondérée de la surface du terrain (30%),
car la surface du terrain influence la valeur totale, surtout pour les maisons.

#### Gestion des valeurs manquantes:
- `.fillna(0)` remplace les éventuelles valeurs manquantes de Surface terrain par 0,
évitant ainsi une division par NaN ou une erreur lors du calcul.



In [ ]:
colonnes_a_garder = [
    'Surface reelle bati',
    'Nombre pieces principales',
    'Nombre de lots',
    'Valeur fonciere',
    'Surface terrain',
    'Code type local'
]

# logements
df_logements = df_4p[colonnes_a_garder].copy()

#Calcul du prix au m² pondéré (surface bâtie + 0.3 * surface terrain) 
df_logements['prix_m2'] = df_logements['Valeur fonciere'] / df_logements['Surface reelle bati']

### Vérifier les types

In [ ]:
df_logements.dtypes

In [ ]:
# Remplacer Surface terrain NaN par 0 pour les appartements (Code type local == 2)
df_logements.loc[
    (df_logements["Code type local"] == 2) & (df_logements["Surface terrain"].isna()),
    "Surface terrain"
] = 0

# Remplacer Surface terrain NaN par 0 pour les maisons (Code type local == 1)
df_logements.loc[
    (df_logements["Code type local"] == 1) & (df_logements["Surface terrain"].isna()),
    "Surface terrain"
] = 0


#mettre surface terrain à 0 pour les appartements
#df_logements.loc[df_logements["Code type local"] == 2, "Surface terrain"] = 0

#mettre le nombre de lots pour les maison à 0
#df_logements.loc[df_logements["Code type local"] == 1, "Nombre de lots"] = 0

df_logements = df_logements.dropna()

print(len(df_logements))

In [ ]:
#séparation du dataset
df_maison = df_logements[df_logements["Code type local"] == 1]
df_appartement = df_logements[df_logements["Code type local"] == 2]

In [ ]:
def detect_outliers(df, column):
    """
    Détecte les outliers d'une colonne numérique d'un DataFrame en utilisant la méthode IQR.
    Retourne les bornes et les lignes considérées comme outliers.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return lower_bound, upper_bound, outliers

In [ ]:
# Pour df_logements
print("📦 Outliers Surface reelle bati (logements) :")
print(detect_outliers(df_maison, "prix_m2")[2], "\n")

print("📦 Outliers Valeur fonciere (logements) :")
print(detect_outliers(df_appartement, "prix_m2")[2], "\n")

In [ ]:
def remove_outliers(df, column):
    lower, upper, _ = detect_outliers(df, column)
    return df[(df[column] >= lower) & (df[column] <= upper)]

In [ ]:
df_maison_clean = remove_outliers(df_maison, "Nombre de lots")
df_appartement_clean = remove_outliers(df_appartement, "Nombre de lots")

df_maison_clean = remove_outliers(df_maison_clean, "Surface terrain")
df_appartement_clean = remove_outliers(df_appartement_clean, "Surface terrain")

df_maison_clean = remove_outliers(df_maison_clean, "Surface reelle bati")
df_appartement_clean = remove_outliers(df_appartement_clean, "Surface reelle bati")


print(len(df_maison_clean))
print(len(df_appartement_clean))

In [ ]:
# Créer la figure et les 2 sous-graphiques côte à côte
fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# --- 1. Histogramme pour les maisons ---
axs[0].hist(df_maison_clean['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[0].axvline(np.median(df_maison_clean['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[0].set_title("Maisons - prix_m2")
axs[0].set_xlabel("prix_m2")
axs[0].set_ylabel("Nombre de biens")
axs[0].grid(True)
axs[0].legend()

# --- 2. Histogramme pour les appartements ---
axs[1].hist(df_appartement_clean['prix_m2'], bins=30, color='orange', edgecolor='w')
axs[1].axvline(np.median(df_appartement_clean['prix_m2']), color='red', linestyle='--', label='Médiane')
axs[1].set_title("Appartements - prix_m2")
axs[1].set_xlabel("prix_m2")
axs[1].grid(True)
axs[1].legend()

# Ajuster l'espacement
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Créer la figure et les 2 sous-graphiques côte à côte
fig, axs = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

# --- 1. Boxplot pour les maisons ---
axs[0].boxplot(df_maison_clean['prix_m2'], vert=False)
axs[0].set_title("Maisons - prix_m2")
axs[0].set_xlabel("prix_m2")
axs[0].grid(True)

# --- 2. Boxplot pour les appartements ---
axs[1].boxplot(df_appartement_clean['prix_m2'], vert=False)
axs[1].set_title("Appartements - prix_m2")
axs[1].set_xlabel("prix_m2")
axs[1].grid(True)

# Ajuster l'espacement
plt.tight_layout()
plt.show()


In [ ]:
# matrices de corrélation
corr_maison = df_maison_clean[["prix_m2", "Nombre de lots",
                        "Surface terrain", "Surface reelle bati"]].corr()
corr_appartement = df_appartement_clean[["prix_m2", "Nombre de lots",
                        "Surface terrain", "Surface reelle bati"]].corr()

# Création des sous-figures
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap pour les maisons
sns.heatmap(corr_maison, ax=axs[0], cmap='Oranges', annot=True, fmt=".2f")
axs[0].set_title("Corrélation - Maisons")

# Heatmap pour les appartements
sns.heatmap(corr_appartement, ax=axs[1], cmap='Oranges', annot=True, fmt=".2f")
axs[1].set_title("Corrélation - Appartements")

plt.tight_layout()
plt.show()

In [ ]:
# Créer la figure avec 2 sous-graphes
fig, axs = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# --- Scatter plot pour les maisons ---
axs[0].scatter(df_maison_clean['Surface reelle bati'], df_maison_clean['Valeur fonciere'], color='blue', alpha=0.5)
axs[0].set_title("Maisons - Surface vs Prix")
axs[0].set_xlabel("Surface (m²)")
axs[0].set_ylabel("Prix (€)")
axs[0].grid(True)

# --- Scatter plot pour les appartements ---
axs[1].scatter(df_appartement_clean['Surface reelle bati'], df_appartement_clean['Valeur fonciere'], color='green', alpha=0.5)
axs[1].set_title("Appartements - Surface vs Prix")
axs[1].set_xlabel("Surface (m²)")
axs[1].grid(True)

# Ajuster l'affichage
plt.tight_layout()
plt.show()

In [ ]:

results_maisons = []
results_appartements = []

# Variables explicatives
features_maisons = ["Nombre de lots", "Surface terrain", "Surface reelle bati"]

features_appartements = ["Nombre de lots", "Surface terrain", "Surface reelle bati"]

X_maison = df_maison[features_maisons]
X_appartement = df_appartement[features_appartements]

# Variable cible
y_maison = df_maison["prix_m2"]
y_appartement = df_appartement["prix_m2"]


# Division en jeu d'entraînement (80%) et test (20%)
X_train_maison, X_test_maison, y_train_maison, y_test_maison = train_test_split(X_maison, y_maison, test_size=0.2, random_state=42)
X_train_appartement, X_test_appartement, y_train_appartement, y_test_appartement = train_test_split(X_appartement, y_appartement, test_size=0.2, random_state=42)


# Optimisation DecisionTreeRegressor
param_dt = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": [2, 5, 10]
}

param_rf = {
    "n_estimators": [50, 100],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5]
}

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "GridSearchCV DT": GridSearchCV(DecisionTreeRegressor(random_state=42), param_dt, cv=5, scoring='neg_mean_squared_error'),
    "GridSearchCV RF": GridSearchCV(RandomForestRegressor(random_state=42), param_rf, cv=5, scoring='neg_mean_squared_error'),  # Ajout du RF optimisé
    "XGBoost": XGBRegressor(random_state=42)
}

# fonction pour pipeline

def pipeline_result(models, X_train, y_train, X_test, y_test):
    results = []
    for name, model in models.items():
        
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)              
        results.append({"Modele": name, "MSE": mse, "RMSE": rmse, "MAE": mae, "R2 Score": r2})
    return results , pipeline

# pipeline maison + appartement
pipeline_result_maison = pipeline_result(models, X_train_maison, y_train_maison, X_test_maison, y_test_maison)
pipeline_result_appartement = pipeline_result(models, X_train_appartement, y_train_appartement, X_test_appartement, y_test_appartement)

results_maisons = pipeline_result_maison[0]
results_appartements = pipeline_result_appartement[0]

# 🔹 Créer les DataFrames pour chaque type de bien
df_resultats_maisons = pd.DataFrame(results_maisons)
df_resultats_appartements = pd.DataFrame(results_appartements)

# 🔹 Trier par MSE croissant (meilleur modèle en haut)
df_resultats_maisons = df_resultats_maisons.sort_values(by="MSE", ascending=True)
df_resultats_appartements = df_resultats_appartements.sort_values(by="MSE", ascending=True)

# 🔹 Afficher les tableaux
print("🏠 Tableau comparatif - MAISONS :")
display(df_resultats_maisons[["Modele", "MSE", "RMSE", "MAE", "R2 Score"]])

print("\n🏢 Tableau comparatif - APPARTEMENTS :")
display(df_resultats_appartements[["Modele", "MSE", "RMSE", "MAE", "R2 Score"]])
    
def plot_predictions(models, type,X_train, y_train, X_test, y_test, titre):
    n_models = len(models)
    n_cols = 2  # fixe à 2 colonnes
    n_rows = math.ceil(n_models / n_cols)  # calcule le nombre de lignes

    plt.figure(figsize=(7 * n_cols, 5 * n_rows))

    for i, (name, model) in enumerate(models.items()):
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        pipeline.fit(X_train, y_train)
        
        os.makedirs(f"../models/{type}", exist_ok=True)
        #sauvegarde modele
        joblib.dump(pipeline, f"../models/{type}/pipeline_{type}_{name}.pkl")
        y_pred = pipeline.predict(X_test)

        plt.subplot(n_rows, n_cols, i + 1)
        plt.scatter(y_test, y_pred, alpha=0.6, edgecolor='k')
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        plt.xlabel("Valeurs réelles (prix/m²)")
        plt.ylabel("Valeurs prédites (prix/m²)")
        plt.title(name)
        plt.grid(True)
        

    plt.suptitle(titre, fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# 📈 Graphe pour les maisons
plot_predictions(models, "maison",X_train_maison, y_train_maison, X_test_maison, y_test_maison, "🏠 Prédictions pour les maisons")

# 📈 Graphe pour les appartements
plot_predictions(models, "appartement",X_train_appartement, y_train_appartement, X_test_appartement, y_test_appartement, "🏢 Prédictions pour les appartements")

In [ ]:
import json

result = {
    "maison": results_maisons,
    "appartement": results_appartements
}

# Sauvegarde des données dans un fichier JSON
with open("resultats_modeles_maison_appartement.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)



# # Exemple : on récupère le pipeline final pour les maisons
# results_maison, best_pipeline_maison = pipeline_result(models, X_train_maison, y_train_maison, X_test_maison, y_test_maison)

# # Et pour les appartements
# results_appartement, best_pipeline_appartement = pipeline_result(models, X_train_appartement, y_train_appartement, X_test_appartement, y_test_appartement)

# # Sauvegarde des pipelines dans des fichiers .pkl
# joblib.dump(best_pipeline_maison, 'pipeline_maison.pkl')
# joblib.dump(best_pipeline_appartement, 'pipeline_appartement.pkl')   
    
# #sauvegarde pipeline maison
# joblib.dump(results_maisons, f"../models/pipeline_maison.joblib")

# #sauvegarde pipeline appartement
# joblib.dump(results_appartements, f"../models/pipeline_appartement.joblib")
